# process memotion dataset

### 1. remove crashed images...

In [4]:
import pandas as pd
import os
import cv2
import numpy as np
from PIL import Image
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

home = '/home/chen_zhang06/HatefulMemes/'

In [5]:
def preprocess_text(memo):
    import re
    from collections import Counter

    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    text_to_remove = ["imgflip.com", "imgflip", "quickmeme.com", "quickmeme",
                      "memecenter.com", "memecenter", "memegenerator.net",
                      "memegenerator", "9gag.com", "arhtisticlicense.com",
                      "starecat.com", "gapbagap.net", "dudelol.com"]

    pat = r'\b(?:{})\b'.format('|'.join(text_to_remove))

    memo.text = memo.text.str.lower()
    # Remove URLs
    memo.text = memo.text.str.replace(url_pattern, "")
    # Remove words in 'text_to_remove'
    memo.text = memo.text.str.replace(pat, '')
    # Remove any character that's not a letter or a number
    memo.text = memo.text.replace(r"[\W_]+", " ", regex=True)
    memo = memo.dropna()
    return memo



In [19]:
# getting csv and image data
path2img = os.path.join(home, "memotion_dataset_7k/images/")
path2data = os.path.join(home, "memotion_dataset_7k/labels.csv")
df = pd.read_csv(path2data, delimiter=',')
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns: \n {list(df.columns)}\n')
print('--------------', df["offensive"].unique())
# print(df.head)

for i in df["offensive"].unique():
    print(f"# of '{i}' memes \t: {len(df[df['offensive']==i])}")

columns = ["image_name", "text_corrected", 'offensive']
# hateful_offensive = df[df["offensive"]=="hateful_offensive"][columns]
hateful_offensive = df[df["offensive"].isin(df["offensive"].unique())][columns]
# print(hateful_offensive)
# very_offensive    = df[df["offensive"]=="very_offensive"][columns]
# not_offensive          = df[df["offensive"]=="not_offensive"][columns]

memo = df[columns]
memo.columns = ["id", "text", 'label']
print(memo)


There are 6992 rows and 9 columns: 
 ['Unnamed: 0', 'image_name', 'text_ocr', 'text_corrected', 'humour', 'sarcasm', 'offensive', 'motivational', 'overall_sentiment']

-------------- ['not_offensive' 'very_offensive' 'slight' 'hateful_offensive']
# of 'not_offensive' memes 	: 2713
# of 'very_offensive' memes 	: 1466
# of 'slight' memes 	: 2592
# of 'hateful_offensive' memes 	: 221
                  id                                               text  \
0        image_1.jpg  LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...   
1       image_2.jpeg  The best of #10 YearChallenge! Completed in le...   
2        image_3.JPG  Sam Thorne @Strippin ( Follow Follow Saw every...   
3        image_4.png              10 Year Challenge - Sweet Dee Edition   
4        image_5.png  10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...   
...              ...                                                ...   
6987  image_6988.jpg  Tuesday is Mardi Gras Wednesday is Valentine's...   
6988  image_6989

In [20]:
# locate empty or mismatched files: 
img_dir = os.path.join(home, 'memotion_dataset_7k/images/')
emptyfile = []
for i in memo['id']: 
    try: 
        path = os.path.join(img_dir, i)
        img = Image.open(path)
        im = np.array(img).astype(np.float32)
    except: 
        emptyfile.append(i)
print(len(emptyfile))

/opt/conda/lib/python3.7/site-packages/PIL/TiffImagePlugin.py:793: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


330


In [23]:
# remove those lines in the pd file, and remove the 'slight' offensive category, as it might be confusing
print(memo.shape)
memo.drop(memo[memo['id'].isin(emptyfile)].index, inplace = True)
memo.drop(memo[memo['label'] == 'slight'].index, inplace = True)
print(memo.shape)

(4155, 3)
(4155, 3)


### 2. Re-assign label values, move those images to another folder

In [ ]:
# then re-assign the label value: 
hatefullist = ['hateful_offensive', 'very_offensive']
# nonhatefullist = ['not_offensive', 'slight', 'very_offensive']
nonhatefullist = ['not_offensive']
memo.loc[memo['label'].isin(hatefullist), 'label'] = 1
memo.loc[memo['label'].isin(nonhatefullist), 'label'] = 0

# Add missing columns so it looks like Hateful Memes data
memo["img"] = ("img/" + memo["id"])
memo["img"] = memo["img"].str.lower()
# remove extension in "image_name" & get the number of the image & add 100k, cause the HM data ID's goes up to 99k
memo["id"] = memo["id"].str.split(".").str.get(0).str.split("_").str.get(1).astype(int) + 100_000
# Re-order columns
memo = memo[["id", "img", 'label', "text"]]

In [25]:
# Pre-process the text in memes
memo = preprocess_text(memo)
print(memo)

          id                 img label  \
0     100001     img/image_1.jpg     0   
1     100002    img/image_2.jpeg     0   
2     100003     img/image_3.jpg     0   
3     100004     img/image_4.png     1   
4     100005     img/image_5.png     1   
...      ...                 ...   ...   
6986  106987  img/image_6987.jpg     0   
6987  106988  img/image_6988.jpg     1   
6988  106989  img/image_6989.jpg     0   
6990  106991  img/image_6991.jpg     0   
6991  106992  img/image_6992.jpg     0   

                                                   text  
0     look there my friend lightyear now all sohalik...  
1     the best of 10 yearchallenge completed in less...  
2     sam thorne strippin follow follow saw everyone...  
3                   10 year challenge sweet dee edition  
4     10 year challenge with no filter 47 hilarious ...  
...                                                 ...  
6986  if you re going on and on and on about your pr...  
6987  tuesday is mardi gras wed

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: The default value of regex will change from True to False in a future version.


In [31]:
# copy these files into another folder: memotion_dataset_7k/images_selected/
import zipfile
flist = list(memo['img'].str.split('/').str.get(1))
# print(flist)

import os
os.chdir(os.path.join(home, 'memotion_dataset_7k/images/'))
with zipfile.ZipFile('memotion_selected.zip', 'w') as zipMe:        
    for file in flist:
        try: 
            zipMe.write(file, compress_type=zipfile.ZIP_DEFLATED)
        except: pass
!mv memotion_selected.zip ../images_selected/

In [ ]:
os.chdir(os.path.join(home, 'memotion_dataset_7k/images_selected/'))
!unzip memotion_selected.zip
# !rm -rf memotion_selected.zip

In [33]:
!rm -rf memotion_selected.zip

In [34]:
# check number of images: 4138
!ls -F | wc -l

4138


### 3. generate jsonl file

In [36]:
print(memo)

          id                 img label  \
0     100001     img/image_1.jpg     0   
1     100002    img/image_2.jpeg     0   
2     100003     img/image_3.jpg     0   
3     100004     img/image_4.png     1   
4     100005     img/image_5.png     1   
...      ...                 ...   ...   
6986  106987  img/image_6987.jpg     0   
6987  106988  img/image_6988.jpg     1   
6988  106989  img/image_6989.jpg     0   
6990  106991  img/image_6991.jpg     0   
6991  106992  img/image_6992.jpg     0   

                                                   text  
0     look there my friend lightyear now all sohalik...  
1     the best of 10 yearchallenge completed in less...  
2     sam thorne strippin follow follow saw everyone...  
3                   10 year challenge sweet dee edition  
4     10 year challenge with no filter 47 hilarious ...  
...                                                 ...  
6986  if you re going on and on and on about your pr...  
6987  tuesday is mardi gras wed

In [38]:
data = memo.to_json(orient='records', lines=True)
with open(os.path.join(home, "annotations/label_memotion.jsonl"), "w", encoding='utf-8') as f:
    f.write(data)